In [ ]:
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from selenium import webdriver
from IPython.display import display, Image, clear_output

In [ ]:
link = 'https://stock-pictures.netlify.app/'

In [ ]:
driver = webdriver.Chrome()

In [ ]:
driver.get(link)

In [ ]:
allImgs = []
while(len(allImgs)==0):
    sleep(1)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    allImgs = soup.find_all('div',class_='container')

In [ ]:
for i in tqdm(range(0,500000,1000)):
    driver.execute_script('window.scrollTo(0,'+str(i)+')')
    sleep(.1)
sleep(.2)
soup = BeautifulSoup(driver.page_source,'html.parser')
allImgs = soup.find_all('div',class_='container')

In [ ]:
data = []
for img in tqdm(allImgs):
    try:
        src = img.find('img',class_='source-img').get('src')
    except:
        src=None
    try:
        tags = img.find('span',class_='tag-color').text 
        tags = tags.split(' - ')[1].split(', ')
    except:
        tags=[]
    try:
        l_c = img.find('div',class_='likes-comments').find_all('span')
        likes = int(l_c[0].text.split()[0])
        comments = int(l_c[1].text.split()[0])
    except:
        likes=np.nan
        comments=np.nan
    data.append((src,tags,likes,comments))

In [ ]:
df = pd.DataFrame(data,columns=['Img-Source','tags','likes','comments'])

In [ ]:
tag = input('Enter a word for images: ')
tag = tag[0].upper()+tag[1:len(tag)].lower()
req = df.loc[df['tags'].apply(lambda l:tag in l),:]
if(len(req)>3):
    req=req.sample(3)
req.reset_index(drop=True,inplace=True)
print('loading ',end='')
for i in range(5):
    print('. ',end='')
    sleep(0.5)
clear_output()
if(len(req)==0):
    print('No Images Found!\nPlease try again with different keyword')
for i in range(3):
    try:
        img = Image(url=req['Img-Source'][i],width=300,height=300)
        display(img)
    except:
        pass

In [ ]:
# max_likes_index = df['likes'].astype(int).idxmax()
# maxLiked = df.loc[max_likes_index,'Img-Source']
# with open('liked.jpg','wb') as fd:
#     res = requests.get(maxLiked)
#     fd.write(res.content)